In [1]:
import os, pickle, random
import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter

In [2]:
os.chdir('..')
from src.models.genetic_algorithm import *
from src.models.train_animation_predictor import *
from src.models import config
from src.models.ordinal_classifier_fnn import *
from src.models.genetic_algorithm import create_animation_vector
from src.animations.insert_animation import create_animated_svg
from src.models.entmoot_functions import *

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load test data for evaluation

In [63]:
test_data = pd.read_csv('data/model_1/model_1_test.csv')

In [64]:
test_data.head()

,filename,animation_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,stroke_width,opacity,stroke_opacity,fill_r,fill_g,fill_b,stroke_r,stroke_g,stroke_b,svg_fill_r,diff_fill_r,svg_fill_g,diff_fill_g,svg_fill_b,diff_fill_b,svg_stroke_r,diff_stroke_r,svg_stroke_g,diff_stroke_g,svg_stroke_b,diff_stroke_b,rel_width,rel_height,rel_x_position,rel_y_position,nr_paths_svg
0,logo_147,10,0.288970,-7.284798,1.094182,-4.437811,-0.352077,-0.933593,-0.200254,0.931484,0.506142,0.100814,0,1.0,1.0,157,157,156,157,157,156,136.230769,20.769231,142.846154,14.153846,154.769231,1.230769,15.461538,141.538462,22.076923,134.923077,34.769231,121.230769,0.017877,0.111550,0.168001,0.084745,13
1,logo_147,0,-2.807291,-11.223132,3.151704,2.541029,-0.119346,0.909742,2.923519,-1.477561,0.059558,-1.025471,0,1.0,1.0,157,157,156,0,0,0,136.230769,20.769231,142.846154,14.153846,154.769231,1.230769,15.461538,-15.461538,22.076923,-22.076923,34.769231,-34.769231,0.077887,0.707306,0.038944,0.562185,13
2,logo_147,1,0.869143,-11.525720,-0.201577,3.906369,-2.661400,-0.757390,-0.195925,-0.380663,-0.033850,0.286074,0,1.0,1.0,157,157,156,0,0,0,136.230769,20.769231,142.846154,14.153846,154.769231,1.230769,15.461538,-15.461538,22.076923,-22.076923,34.769231,-34.769231,0.026691,0.467979,0.161487,0.450085,13
3,logo_147,12,9.519547,-1.598271,-4.577534,-2.522939,-0.754545,-0.054362,-1.478206,-1.002636,-1.703539,-0.243925,0,1.0,1.0,22,65,148,22,65,148,136.230769,-114.230769,142.846154,-77.846154,154.769231,-6.769231,15.461538,6.538462,22.076923,42.923077,34.769231,113.230769,0.057923,0.271332,0.502114,0.135666,13
4,logo_147,2,-0.253583,-11.400860,-0.170891,5.689338,-0.972701,1.972220,-0.119872,0.083583,0.253451,-0.066204,0,1.0,1.0,157,157,156,0,0,0,136.230769,20.769231,142.846154,14.153846,154.769231,1.230769,15.461538,-15.461538,22.076923,-22.076923,34.769231,-34.769231,0.050151,0.477448,0.113555,0.445340,13


In [65]:
test_data = retrieve_m1_predictions(test_data)

2021-04-22 15:48:27::INFO::Number of animated paths: 448/1892


In [66]:
test_data = retrieve_animation_midpoints(test_data, drop=True)

2021-04-22 15:48:28::INFO::Non-animated paths won't be used for training
2021-04-22 15:48:28::INFO::Start extraction midpoint of animated paths as feature


In [67]:
scaler = pickle.load(open(config.scaler_path, 'rb'))
test_data[config.sm_features] = scaler.transform(test_data[config.sm_features])

In [68]:
test_data.head()

,filename,animation_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,stroke_width,opacity,stroke_opacity,fill_r,fill_g,fill_b,stroke_r,stroke_g,stroke_b,svg_fill_r,diff_fill_r,svg_fill_g,diff_fill_g,svg_fill_b,diff_fill_b,svg_stroke_r,diff_stroke_r,svg_stroke_g,diff_stroke_g,svg_stroke_b,diff_stroke_b,rel_width,rel_height,rel_x_position,rel_y_position,nr_paths_svg,animated,animated_animation_ids,rel_position_to_animations,rel_x_position_to_animations,rel_y_position_to_animations
0,logo_147,10,-0.571819,-1.426890,0.383585,-1.004616,0.378244,-1.071720,-0.384838,0.366493,0.308681,0.062218,0,1.0,1.0,0.042371,0.328460,0.461983,157,157,156,0.248085,-0.152754,0.460976,-0.018242,0.693202,-0.047933,15.461538,141.538462,22.076923,134.923077,34.769231,121.230769,-1.021080,-0.822220,-0.998481,-1.669510,-0.578179,1,"[10, 12, 11]","(0.0428012672222377, 0.08474520367214555)",-1.818319,-1.798633
1,logo_147,12,0.593413,-0.437554,-0.729631,-0.622965,0.232627,-0.737128,-1.199584,-0.493894,-0.990289,-0.222805,0,1.0,1.0,-1.261968,-0.726501,0.374206,22,65,148,0.248085,-1.579739,0.460976,-1.176625,0.693202,-0.151141,15.461538,6.538462,22.076923,42.923077,34.769231,113.230769,-0.847437,-0.239493,0.415467,-1.416171,-0.578179,1,"[10, 12, 11]","(0.923646143983946, 0.13566598867571875)",1.677861,-1.574572
2,logo_147,11,0.481299,-1.005973,-0.524167,1.329646,0.299338,-0.301811,-1.634655,-0.480645,-0.652586,-0.726003,0,1.0,1.0,-1.261968,-0.726501,0.374206,22,65,148,0.248085,-1.579739,0.460976,-1.176625,0.693202,-0.151141,15.461538,6.538462,22.076923,42.923077,34.769231,113.230769,-0.312756,1.687040,0.044134,0.895005,-0.578179,1,"[10, 12, 11]","(0.6923176974460455, 0.6002088945082733)",0.759691,0.469503
3,logo_148,11,-0.753985,-1.719226,0.661278,0.786998,-0.369005,-1.743223,-0.790624,0.585541,-1.526946,-0.038738,0,1.0,1.0,0.042371,0.328460,0.461983,0,0,0,-1.002370,0.850840,-0.530179,0.796444,0.608083,0.024652,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.015033,0.859234,-1.180681,-0.666743,-0.522421,1,[11],"(0.4999999999999999, 0.5)",-0.003642,0.028565
4,logo_149,11,0.135500,0.300492,-0.928966,0.829365,-0.883971,-1.672449,-1.347737,1.380280,-2.470160,-0.770451,0,1.0,1.0,0.042371,0.328460,0.461983,0,0,0,-1.002370,0.850840,-0.530179,0.796444,0.608083,0.024652,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.524704,0.044769,0.408603,-1.222279,-0.522421,1,"[11, 12, 13]","(0.650810561967709, 0.2862981256047344)",0.594944,-0.911763


In [69]:
test_data.to_csv('data/entmoot_optimization/evaluation_data.csv', index=False)

In [70]:
len(test_data)

448

In [71]:
path_vectors = test_data[config.sm_features].values.tolist()

## Apply Entmoot optimizer to test data

In [72]:
with open("models/entmoot_optimizer.pkl", "rb") as f:
    optimizer = pickle.load(f)

In [73]:
# Load surrogate model for function evaluations
func = SurrogateModelFNN()

In [74]:
# Predict animation vectors
an_vec_preds = []
score_preds = []
for i in range(len(path_vectors)):
    opt_x, opt_y = entmoot_predict(optimizer, func, path_vectors[i])
    an_vec_preds.append(opt_x)
    score_preds.append(-opt_y)
    if (i+1) % 10 == 0:
        print(f'Finished animation prediction for path {i+1}.')

Finished animation prediction for path 10.
Finished animation prediction for path 20.
Finished animation prediction for path 30.
Finished animation prediction for path 40.
Finished animation prediction for path 50.
Finished animation prediction for path 60.
Finished animation prediction for path 70.
Finished animation prediction for path 80.
Finished animation prediction for path 90.
Finished animation prediction for path 100.
Finished animation prediction for path 110.
Finished animation prediction for path 120.
Finished animation prediction for path 130.
Finished animation prediction for path 140.
Finished animation prediction for path 150.
Finished animation prediction for path 160.
Finished animation prediction for path 170.
Finished animation prediction for path 180.
Finished animation prediction for path 190.
Finished animation prediction for path 200.
Finished animation prediction for path 210.
Finished animation prediction for path 220.
Finished animation prediction for path 23

In [75]:
#animation_vectors = [create_animation_vector(predictions[i]).tolist() for i in range(predictions.shape[0])]
#test_data['animation_vector'].iloc[0:20] = an_vec_preds
test_data['animation_vector'] = an_vec_preds

In [76]:
test_data.head()

,filename,animation_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,stroke_width,opacity,stroke_opacity,fill_r,fill_g,fill_b,stroke_r,stroke_g,stroke_b,svg_fill_r,diff_fill_r,svg_fill_g,diff_fill_g,svg_fill_b,diff_fill_b,svg_stroke_r,diff_stroke_r,svg_stroke_g,diff_stroke_g,svg_stroke_b,diff_stroke_b,rel_width,rel_height,rel_x_position,rel_y_position,nr_paths_svg,animated,animated_animation_ids,rel_position_to_animations,rel_x_position_to_animations,rel_y_position_to_animations,animation_vector
0,logo_147,10,-0.571819,-1.426890,0.383585,-1.004616,0.378244,-1.071720,-0.384838,0.366493,0.308681,0.062218,0,1.0,1.0,0.042371,0.328460,0.461983,157,157,156,0.248085,-0.152754,0.460976,-0.018242,0.693202,-0.047933,15.461538,141.538462,22.076923,134.923077,34.769231,121.230769,-1.021080,-0.822220,-0.998481,-1.669510,-0.578179,1,"[10, 12, 11]","(0.0428012672222377, 0.08474520367214555)",-1.818319,-1.798633,"[0, 1, 0, 0, 0, 0, 0.0, 0.0, 0.134364244112401..."
1,logo_147,12,0.593413,-0.437554,-0.729631,-0.622965,0.232627,-0.737128,-1.199584,-0.493894,-0.990289,-0.222805,0,1.0,1.0,-1.261968,-0.726501,0.374206,22,65,148,0.248085,-1.579739,0.460976,-1.176625,0.693202,-0.151141,15.461538,6.538462,22.076923,42.923077,34.769231,113.230769,-0.847437,-0.239493,0.415467,-1.416171,-0.578179,1,"[10, 12, 11]","(0.923646143983946, 0.13566598867571875)",1.677861,-1.574572,"[0, 1, 0, 0, 0, 0, 0.0, 0.0, 0.396058242610681..."
2,logo_147,11,0.481299,-1.005973,-0.524167,1.329646,0.299338,-0.301811,-1.634655,-0.480645,-0.652586,-0.726003,0,1.0,1.0,-1.261968,-0.726501,0.374206,22,65,148,0.248085,-1.579739,0.460976,-1.176625,0.693202,-0.151141,15.461538,6.538462,22.076923,42.923077,34.769231,113.230769,-0.312756,1.687040,0.044134,0.895005,-0.578179,1,"[10, 12, 11]","(0.6923176974460455, 0.6002088945082733)",0.759691,0.469503,"[0, 0, 0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
3,logo_148,11,-0.753985,-1.719226,0.661278,0.786998,-0.369005,-1.743223,-0.790624,0.585541,-1.526946,-0.038738,0,1.0,1.0,0.042371,0.328460,0.461983,0,0,0,-1.002370,0.850840,-0.530179,0.796444,0.608083,0.024652,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.015033,0.859234,-1.180681,-0.666743,-0.522421,1,[11],"(0.4999999999999999, 0.5)",-0.003642,0.028565,"[0, 0, 0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
4,logo_149,11,0.135500,0.300492,-0.928966,0.829365,-0.883971,-1.672449,-1.347737,1.380280,-2.470160,-0.770451,0,1.0,1.0,0.042371,0.328460,0.461983,0,0,0,-1.002370,0.850840,-0.530179,0.796444,0.608083,0.024652,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.524704,0.044769,0.408603,-1.222279,-0.522421,1,"[11, 12, 13]","(0.650810561967709, 0.2862981256047344)",0.594944,-0.911763,"[0, 1, 0, 0, 0, 0, 0.0, 0.0, 0.396058242610681..."


In [77]:
test_data['reward'] = score_preds

In [78]:
test_data.head()

,filename,animation_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,stroke_width,opacity,stroke_opacity,fill_r,fill_g,fill_b,stroke_r,stroke_g,stroke_b,svg_fill_r,diff_fill_r,svg_fill_g,diff_fill_g,svg_fill_b,diff_fill_b,svg_stroke_r,diff_stroke_r,svg_stroke_g,diff_stroke_g,svg_stroke_b,diff_stroke_b,rel_width,rel_height,rel_x_position,rel_y_position,nr_paths_svg,animated,animated_animation_ids,rel_position_to_animations,rel_x_position_to_animations,rel_y_position_to_animations,animation_vector,reward
0,logo_147,10,-0.571819,-1.426890,0.383585,-1.004616,0.378244,-1.071720,-0.384838,0.366493,0.308681,0.062218,0,1.0,1.0,0.042371,0.328460,0.461983,157,157,156,0.248085,-0.152754,0.460976,-0.018242,0.693202,-0.047933,15.461538,141.538462,22.076923,134.923077,34.769231,121.230769,-1.021080,-0.822220,-0.998481,-1.669510,-0.578179,1,"[10, 12, 11]","(0.0428012672222377, 0.08474520367214555)",-1.818319,-1.798633,"[0, 1, 0, 0, 0, 0, 0.0, 0.0, 0.134364244112401...",2
1,logo_147,12,0.593413,-0.437554,-0.729631,-0.622965,0.232627,-0.737128,-1.199584,-0.493894,-0.990289,-0.222805,0,1.0,1.0,-1.261968,-0.726501,0.374206,22,65,148,0.248085,-1.579739,0.460976,-1.176625,0.693202,-0.151141,15.461538,6.538462,22.076923,42.923077,34.769231,113.230769,-0.847437,-0.239493,0.415467,-1.416171,-0.578179,1,"[10, 12, 11]","(0.923646143983946, 0.13566598867571875)",1.677861,-1.574572,"[0, 1, 0, 0, 0, 0, 0.0, 0.0, 0.396058242610681...",2
2,logo_147,11,0.481299,-1.005973,-0.524167,1.329646,0.299338,-0.301811,-1.634655,-0.480645,-0.652586,-0.726003,0,1.0,1.0,-1.261968,-0.726501,0.374206,22,65,148,0.248085,-1.579739,0.460976,-1.176625,0.693202,-0.151141,15.461538,6.538462,22.076923,42.923077,34.769231,113.230769,-0.312756,1.687040,0.044134,0.895005,-0.578179,1,"[10, 12, 11]","(0.6923176974460455, 0.6002088945082733)",0.759691,0.469503,"[0, 0, 0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",2
3,logo_148,11,-0.753985,-1.719226,0.661278,0.786998,-0.369005,-1.743223,-0.790624,0.585541,-1.526946,-0.038738,0,1.0,1.0,0.042371,0.328460,0.461983,0,0,0,-1.002370,0.850840,-0.530179,0.796444,0.608083,0.024652,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.015033,0.859234,-1.180681,-0.666743,-0.522421,1,[11],"(0.4999999999999999, 0.5)",-0.003642,0.028565,"[0, 0, 0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",2
4,logo_149,11,0.135500,0.300492,-0.928966,0.829365,-0.883971,-1.672449,-1.347737,1.380280,-2.470160,-0.770451,0,1.0,1.0,0.042371,0.328460,0.461983,0,0,0,-1.002370,0.850840,-0.530179,0.796444,0.608083,0.024652,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.524704,0.044769,0.408603,-1.222279,-0.522421,1,"[11, 12, 13]","(0.650810561967709, 0.2862981256047344)",0.594944,-0.911763,"[0, 1, 0, 0, 0, 0, 0.0, 0.0, 0.396058242610681...",2


In [79]:
test_data['reward'].mean()

2.421875

In [80]:
test_data['reward'].value_counts()

2    201
3    185
1     36
4     23
0      3
Name: reward, dtype: int64

In [81]:
def get_type(row):
    return np.argmax(row['animation_vector'][:6])

In [82]:
test_data['type'] = test_data.apply(lambda row: get_type(row), axis=1)
test_data['type'] = test_data['type'].map({0: 'translate', 1: 'scale', 2: 'rotate', 3: 'skew', 4: 'fill', 5: 'opacity'})

In [83]:
test_data['type'].value_counts()

fill         206
scale        110
opacity       94
rotate        34
translate      4
Name: type, dtype: int64

## Create animated SVGs

In [85]:
gb = [test_data.groupby('filename')[column].apply(list) for column in 'animation_id animation_vector reward'.split()]
svg_animations = pd.concat(gb, axis=1).reset_index()

In [86]:
svg_animations.head()

,filename,animation_id,animation_vector,reward
0,logo_147,"[10, 12, 11]","[[0, 1, 0, 0, 0, 0, 0.0, 0.0, 0.13436424411240...","[2, 2, 2]"
1,logo_148,[11],"[[0, 0, 0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",[2]
2,logo_149,"[11, 12, 13]","[[0, 1, 0, 0, 0, 0, 0.0, 0.0, 0.39605824261068...","[2, 2, 2]"
3,logo_150,[11],"[[0, 1, 0, 0, 0, 0, 0.0, 0.0, 0.10961812000000...",[2]
4,logo_151,"[0, 14, 13, 15]","[[0, 1, 0, 0, 0, 0, 0.0, 0.0, 0.18520616700021...","[2, 2, 2, 1]"


In [87]:
len(svg_animations)

87

In [89]:
for i, row in svg_animations.iterrows():
    try:
        rewards = '-'.join([str(reward) for reward in row['reward']])
        ids = '-'.join([str(reward) for reward in row['animation_id']])
        animation_id = f'{ids}---{rewards}'
        create_animated_svg(f"data/svgs/{row['filename']}.svg", row['animation_id'], row['animation_vector'], animation_id)
    except FileNotFoundError:
        print(f"File not found: {row['filename']}")
        pass

File not found: logo_406
